In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter

from sklearn.externals import joblib
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, classification_report


%matplotlib inline
plt.style.use('ggplot')

from utils.clean_utils import reduce_dataframe, clean_dataframe
from utils.model import model_RandomClass

In [2]:
df_clean = pd.read_csv('data/feats_cleaned.csv')
df_reduce = pd.read_csv('data/feats_reduced_byRF.csv')

# Logistic Regression using the Reduced DF

In [3]:
columns_R = df_reduce.columns
feat_cols_R = []
for name in columns_R:
    if name != "structureProteinName" and name != "cellID" and name != "save_feats_path":
        feat_cols_R.append(name)

In [4]:
# Split to features and labels
X_R = df_reduce[feat_cols_R]
y_R = df_reduce.structureProteinName

In [5]:
X_train_R, X_test_R, y_train_R, y_test_R = train_test_split(X_R, y_R, random_state=10)

In [6]:
model_R = LogisticRegression(penalty='l1')

In [7]:
# model_R.fit(X_train_R, y_train_R)

In [8]:
#joblib.dump(model_R, 'models/logregL1_reduced.pkl') 

### Classification Report Comparisons

In [9]:
model_R = joblib.load('models/logregL1_reduced.pkl')

In [10]:
# Read in an L2 model from a different session with a test/train split with same random state
model_R_l2 = joblib.load('models/logregL2_reduced.pkl')

In [11]:
model_R.score(X_test_R, y_test_R)

0.31987520645990092

In [12]:
# L1 LogReg
print(classification_report(y_true=y_test_R, y_pred=model_R.predict(X_test_R)))

               precision    recall  f1-score   support

Alpha actinin       0.21      0.31      0.25       127
Alpha tubulin       0.32      0.31      0.32       852
   Beta actin       0.31      0.25      0.27       414
  Desmoplakin       0.27      0.25      0.26       605
  Fibrillarin       0.23      0.24      0.23       234
     Lamin B1       0.38      0.48      0.42      1105
   Myosin IIB       0.07      0.08      0.07        49
      ST6GAL1       0.35      0.28      0.31       402
   Sec61 beta       0.26      0.14      0.18       499
        Tom20       0.34      0.36      0.35      1102
          ZO1       0.21      0.22      0.21        60

  avg / total       0.32      0.32      0.31      5449



In [13]:
model_R_l2.score(X_test_R,y_test_R)

0.30556065333088639

In [14]:
# L2 LogReg
print(classification_report(y_true=y_test_R, y_pred=model_R_l2.predict(X_test_R)))

               precision    recall  f1-score   support

Alpha actinin       0.25      0.28      0.27       127
Alpha tubulin       0.31      0.31      0.31       852
   Beta actin       0.29      0.22      0.25       414
  Desmoplakin       0.24      0.20      0.22       605
  Fibrillarin       0.21      0.21      0.21       234
     Lamin B1       0.35      0.48      0.41      1105
   Myosin IIB       0.12      0.12      0.12        49
      ST6GAL1       0.33      0.22      0.27       402
   Sec61 beta       0.24      0.12      0.16       499
        Tom20       0.32      0.36      0.34      1102
          ZO1       0.25      0.30      0.27        60

  avg / total       0.30      0.31      0.30      5449



In [39]:
# Compare to baseline of random guessing based on class distributions
print(classification_report(y_true=y_test_R, y_pred=model_RandomClass(y_test_R)))

               precision    recall  f1-score   support

Alpha actinin       0.01      0.02      0.01       127
Alpha tubulin       0.18      0.18      0.18       852
   Beta actin       0.08      0.07      0.08       414
  Desmoplakin       0.11      0.11      0.11       605
  Fibrillarin       0.03      0.03      0.03       234
     Lamin B1       0.21      0.22      0.22      1105
   Myosin IIB       0.00      0.00      0.00        49
      ST6GAL1       0.09      0.08      0.09       402
   Sec61 beta       0.09      0.10      0.10       499
        Tom20       0.21      0.20      0.21      1102
          ZO1       0.00      0.00      0.00        60

  avg / total       0.15      0.15      0.15      5449



### Investigate Coefficients

In [16]:
# Split to features and labels
X_R_notreduced = df_reduce[feat_cols_R]
y_Rn = df_reduce.structureProteinName

In [17]:
# Normalize so coefficients can be compared
min_max_scaler = MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(X_R_notreduced)
df_normalized = pd.DataFrame(np_scaled)
df_normalized.columns = feat_cols_R

In [18]:
X_Rn = df_normalized

In [19]:
X_train_Rn, X_test_Rn, y_train_Rn, y_test_Rn = train_test_split(X_Rn, y_Rn, random_state=10)

In [20]:
model_Rn = LogisticRegression(penalty='l1')

In [21]:
#model_Rn.fit(X_train_Rn, y_train_Rn)

In [22]:
#joblib.dump(model_Rn, 'models/logregL1_reduced_norm.pkl') 

In [23]:
model_Rn = joblib.load('models/logregL1_reduced_norm.pkl')

In [24]:
print(classification_report(y_true=y_test_Rn, y_pred=model_Rn.predict(X_test_Rn)))

               precision    recall  f1-score   support

Alpha actinin       0.31      0.31      0.31       127
Alpha tubulin       0.33      0.34      0.34       852
   Beta actin       0.38      0.28      0.32       414
  Desmoplakin       0.28      0.25      0.27       605
  Fibrillarin       0.30      0.22      0.26       234
     Lamin B1       0.39      0.55      0.45      1105
   Myosin IIB       0.00      0.00      0.00        49
      ST6GAL1       0.41      0.27      0.33       402
   Sec61 beta       0.30      0.11      0.16       499
        Tom20       0.33      0.41      0.37      1102
          ZO1       0.39      0.27      0.32        60

  avg / total       0.34      0.35      0.33      5449



In [25]:
coef_df = pd.DataFrame(np.transpose(model_Rn.coef_))
coef_df.columns = model_Rn.classes_
coefficients = pd.concat([pd.DataFrame(X_Rn.columns),coef_df], axis = 1)

In [26]:
# Look at coefficients for largest class, Lamin B1, where the coef is over 2
coefficients[abs(coefficients["Lamin B1"])>2]\
.sort_values("Lamin B1", ascending=False)[[0,"Lamin B1"]]

,0,Lamin B1
8,feat_nuc_mt_edge_1,4.999023
195,feat_cell_obj_mean_depth,3.598484
13,feat_cell_region_histogram_px_1,3.074944
86,feat_cell_mt_distHist_2,2.978566
689,feat_nuc_mt_edge_region_hist_2,2.623696
98,feat_cell_mt_region_hist_151,2.198621
284,feat_cell_mt_region_hist_2,2.042106
40,feat_nuc_polarity_nuc_distal_affinity,-2.049277
232,feat_cell_obj_tot_sphericity,-2.183667
857,feat_nuc_mt_edge_region_hist_43,-2.304385


# Normalized LogReg w/ balanced class weights

In [34]:
model_Rn_balance = LogisticRegression(penalty='l2', class_weight="balanced")

In [35]:
#model_Rn_balance.fit(X_train_Rn, y_train_Rn)

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [36]:
#joblib.dump(model_Rn_balance, 'models/logregL2_reduced_norm_balance.pkl') 

['models/logregL2_reduced_norm_balance.pkl']

In [37]:
model_Rn_balance = joblib.load('models/logregL2_reduced_norm_balance.pkl')

In [38]:
print(classification_report(y_true=y_test_Rn, y_pred=model_Rn_balance.predict(X_test_Rn)))

               precision    recall  f1-score   support

Alpha actinin       0.24      0.61      0.35       127
Alpha tubulin       0.36      0.27      0.31       852
   Beta actin       0.29      0.34      0.31       414
  Desmoplakin       0.29      0.29      0.29       605
  Fibrillarin       0.22      0.38      0.28       234
     Lamin B1       0.44      0.40      0.42      1105
   Myosin IIB       0.11      0.31      0.16        49
      ST6GAL1       0.32      0.50      0.39       402
   Sec61 beta       0.24      0.16      0.19       499
        Tom20       0.37      0.26      0.30      1102
          ZO1       0.23      0.52      0.32        60

  avg / total       0.34      0.32      0.32      5449



# Permutation Test

In [27]:
print(classification_report(y_true=y_test_Rn, y_pred=model_Rn.predict(X_test_Rn)))

               precision    recall  f1-score   support

Alpha actinin       0.31      0.31      0.31       127
Alpha tubulin       0.33      0.34      0.34       852
   Beta actin       0.38      0.28      0.32       414
  Desmoplakin       0.28      0.25      0.27       605
  Fibrillarin       0.30      0.22      0.26       234
     Lamin B1       0.39      0.55      0.45      1105
   Myosin IIB       0.00      0.00      0.00        49
      ST6GAL1       0.41      0.27      0.33       402
   Sec61 beta       0.30      0.11      0.16       499
        Tom20       0.33      0.41      0.37      1102
          ZO1       0.39      0.27      0.32        60

  avg / total       0.34      0.35      0.33      5449



#### Scramble labels to see how the model performs

In [40]:
print(classification_report(y_true=np.random.choice(y_test_Rn,size=len(y_test_Rn),replace=False), y_pred=model_Rn.predict(X_test_Rn)))

               precision    recall  f1-score   support

Alpha actinin       0.02      0.02      0.02       127
Alpha tubulin       0.14      0.15      0.14       852
   Beta actin       0.06      0.04      0.05       414
  Desmoplakin       0.10      0.09      0.09       605
  Fibrillarin       0.04      0.03      0.03       234
     Lamin B1       0.20      0.28      0.23      1105
   Myosin IIB       0.00      0.00      0.00        49
      ST6GAL1       0.05      0.03      0.04       402
   Sec61 beta       0.12      0.05      0.07       499
        Tom20       0.19      0.23      0.21      1102
          ZO1       0.02      0.02      0.02        60

  avg / total       0.13      0.15      0.14      5449



# Train LogReg on all Features - w/ balanced class weights

In [29]:
columns_clean = df_clean.columns
feat_cols_clean = []
for name in columns_clean:
    if name != "structureProteinName" and name != "cellID" and name != "save_feats_path":
        feat_cols_clean.append(name)

In [41]:
# Split to features and labels
X_clean_temp = df_clean[feat_cols_clean]
y_clean = df_clean.structureProteinName

In [42]:
# Normalize so coefficients can be compared
min_max_scaler = MinMaxScaler()
np_scaled_clean = min_max_scaler.fit_transform(X_clean_temp)
df_normalized_clean = pd.DataFrame(np_scaled_clean)
df_normalized_clean.columns = feat_cols_clean

In [43]:
X_clean = df_normalized_clean

In [44]:
X_train_clean, X_test_clean, y_train_clean, y_test_clean = train_test_split(X_clean, y_clean, random_state=10)

In [45]:
X_train_clean.shape

(16344, 1663)

In [46]:
model_clean = LogisticRegression(penalty='l2', class_weight='balanced')

In [47]:
model_clean.fit(X_train_clean, y_train_clean)

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [48]:
joblib.dump(model_clean, 'models/logregL2_clean_norm_balance.pkl') 

['models/logregL2_clean_norm_balance.pkl']

In [ ]:
#model_clean =joblibblib.load('models/logregL2_clean_norm_balance.pkl')

In [49]:
print(classification_report(y_true=y_test_clean, y_pred=model_clean.predict(X_test_clean)))

               precision    recall  f1-score   support

Alpha actinin       0.26      0.56      0.35       127
Alpha tubulin       0.36      0.28      0.31       852
   Beta actin       0.28      0.35      0.31       414
  Desmoplakin       0.25      0.26      0.26       605
  Fibrillarin       0.22      0.38      0.28       234
     Lamin B1       0.44      0.39      0.42      1105
   Myosin IIB       0.14      0.24      0.18        49
      ST6GAL1       0.32      0.46      0.37       402
   Sec61 beta       0.22      0.18      0.20       499
        Tom20       0.36      0.28      0.31      1102
          ZO1       0.31      0.53      0.39        60

  avg / total       0.33      0.32      0.32      5449



In [ ]:
s